In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import C14preproc as prep
import C14utils
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset
from scipy.interpolate import interp1d
import isamcalc_lib as isam
import os.path
import uq_lib as uq
import SOCtools as soc
from os import path


In [2]:
#================================================================================
# Initilize the site information
#================================================================================
# obs
obs_fname = 'Non_peat_data_permafrost.csv'
data = pd.read_csv(obs_fname,encoding='iso-8859-1',index_col='ProfileID', skiprows=[1])
all_profid = data.index.unique()
lons = prep.getvarxls(data,'Lon',all_profid[0:16],0)
lats = prep.getvarxls(data,'Lat',all_profid[0:16],0)

# Calibration
isam_cal_d14c_fname = 'isam_dc14_cali.dat'
isam_cal_soc_fname = 'isam_soc_cali.dat'

# Um's results
isam_um_d14c_fname = 'isam_um_dc14.dat'
isam_um_soc_fname = 'isam_um_soc.dat'


In [3]:
# Create dictionaries
site = {
    "43"  : 0,
    "110" : 1,
    "143" : 2,
    "197" : 3,
    "146" : 4,   
}

yr_dict = {
    "43"  : 2001,
    "110" : 2005,
    "143" : 2007,
    "197" : 1999,
    "146" : 2002,   
}

sen_exp = {
    "aa_u"      : 0,    # high 
    "aa_l"      : 1,    # low
    "bb_u"      : 2,
    "bb_l"      : 3,
    "bd_u"      : 4,
    "bd_l"      : 5,
    "control"   : 6,
    "d_u"       : 7,
    "d_l"       : 8,
    "dep_u"     : 9,
    "dep_l"     : 10,
    "q_u"       : 12,
    "q_l"       : 13,
    "s_u"       : 14,
    "s_l"       : 15,
    "gamma_u"   : 16,
    "gamma_l"   : 17,
}

In [ ]:
float('NaN')

In [4]:
# Read in observations if available
data.nodedepth = data.Layer_top + (data.Layer_bottom - data.Layer_top)/2.

obs_topsoil = np.zeros((3, 5))
obs_subsoil = np.zeros((3, 5))

for siteid, ind in site.items():
    # Node depth
    dn = data.nodedepth.loc[int(float(siteid))].as_matrix()
    # Layer thickness
    dp = (data.Layer_bottom.loc[int(float(siteid))] - data.Layer_top.loc[int(float(siteid))]).as_matrix()
    # Layer interface
    zsoih = np.concatenate((data.Layer_top.loc[int(float(siteid))].as_matrix()[0:1], data.Layer_bottom.loc[int(float(siteid))].as_matrix()))
    # Convert from cm to m
    zsoih = zsoih/100.
    dp = dp/100.
    # Number of layers of the current profile 
    num_obs = data.C_Density[int(float(siteid))].__len__()
    # SOC
    socobs = data.C_Density.loc[int(float(siteid))].as_matrix() * 1000.
    obs_topsoil[0, ind] = soc.aggre_prof_den_to_stock(0.3, zsoih, dp, socobs)
    obs_subsoil[0, ind] = soc.aggre_prof_den_to_stock(1, zsoih, dp, socobs)
    
    # D14C
    d14cobs = data.D14C_BulkLayer.loc[int(float(siteid))].as_matrix()
    # Calculate SOC content profile (kgC m-2) instead of SOC density profile (kgC m-3)
    soc2obs = socobs*dp
    obs_topsoil[1, ind] = C14utils.avg_soc_weighted_d14c(0.3, zsoih, dp, soc2obs, d14cobs)
    obs_subsoil[1, ind] = C14utils.avg_soc_weighted_d14c(1.0, zsoih, dp, soc2obs, d14cobs)
    
# Calculate the turnover time
sampleyr = [2001, 2005, 2007, 1999, 2002]
tau, cost = C14utils.cal_tau(obs_topsoil[1,:], sampleyr, 1, 0)
obs_topsoil[2, :] = tau[:,0]
tau, cost = C14utils.cal_tau(obs_subsoil[1,:], sampleyr, 1, 0)
obs_subsoil[2, :] = tau[:,0]

/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/numba/dataflow.py:346: RuntimeWarning: Python2 style print partially supported.  Please use Python3 style print.
  "Python3 style print.", RuntimeWarning)



 -----  obs: 0  ------ 



 -----  obs: 1  ------ 



 -----  obs: 2  ------ 



 -----  obs: 3  ------ 



 -----  obs: 4  ------ 



 -----  obs: 0  ------ 



 -----  obs: 1  ------ 



 -----  obs: 2  ------ 



 -----  obs: 3  ------ 



 -----  obs: 4  ------ 




In [29]:
data.nodedepth.loc[int(float(143))].as_matrix()

array([   1.5,    5.5,   11. ,   17.5,   29. ,   44.5,   64.5,   87. ,
        107. ,  125.5,  144. ,  164. ])

In [32]:
obs_topsoil

array([[           nan,    11.05897   ,    12.20739766,            nan,
                   nan],
       [           nan,  -145.04340639,   -97.84353977,            nan,
                   nan],
       [ 2000.        ,  1483.        ,   996.        ,  2000.        ,
         2000.        ]])

In [5]:
#=============================================================
# Obtain the results of sensitivity test for each parameters
#=============================================================

# Read in observations if available
obs_fname = 'Non_peat_data_permafrost.csv'
data = pd.read_csv(obs_fname,encoding='iso-8859-1',index_col='ProfileID', skiprows=[1])
all_profid = data.index.unique()
lons = prep.getvarxls(data,'Lon',all_profid[0:16],0)
lats = prep.getvarxls(data,'Lat',all_profid[0:16],0)

data.nodedepth = data.Layer_top + (data.Layer_bottom - data.Layer_top)/2.
for siteid, ind in site.items():
    d14co = data.D14C_BulkLayer.loc[int(float(siteid))]
    nd = data.nodedepth.loc[int(float(siteid))]
    if(d14co.__class__.__name__ == 'float64'):
        Xobs = d14co   # SOC profile kgCm-3
        Yobs = nd      # 1cm to 200cm
    else:
        Xobs = d14co.as_matrix()   # SOC profile kgCm-3
        Yobs = nd.as_matrix()     # 1cm to 200cm

# Read in model output from the sensitivity test cases
#initialize
soccm_test = [None] * 5
d14cm_test = [None] * 5
bulk_d14c_topsoil = np.zeros((18, 5))
tau_topsoil = np.zeros((18, 5))
bulk_d14c_subsoil = np.zeros((18, 5))
tau_subsoil = np.zeros((18, 5))

for siteid, ind in site.items():
    fname = 'isam_soc_'+str(siteid)+'sen.dat'
    soccm_test[ind] = pd.read_table(fname, header=None, delimiter=r"\s+")
    soccm_test[ind].columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
    soccm_test[ind] = soccm_test[ind].set_index('ID')
    fname = 'isam_dc14_'+str(siteid)+'sen.dat'
    d14cm_test[ind] = pd.read_table(fname, header=None, delimiter=r"\s+")
    d14cm_test[ind].columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
    d14cm_test[ind] = d14cm_test[ind].set_index('ID')

# Calculate tau for each case
for siteid, ind in site.items():
    # Read in D14C of the ISAM model output
    fname = 'isam_dc14_'+str(siteid)+'sen.dat'
    d14cm_test[ind] = pd.read_table(fname, header=None, delimiter=r"\s+")
    d14cm_test[ind].columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
    d14cm_test[ind] = d14cm_test[ind].set_index('ID')
    # Read in SOC
    fname = 'isam_soc_'+str(siteid)+'sen.dat'
    soccm_test[ind] = pd.read_table(fname, header=None, delimiter=r"\s+")
    soccm_test[ind].columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
    soccm_test[ind] = soccm_test[ind].set_index('ID')

    # Weighted C14 for topsoil and subsoil separatedly
    # Totally 18 cases for each site
    sampleyr = yr_dict[str(siteid)] * np.ones(len(d14cm_test[ind]))
    for i in np.arange(0,18):
        bulk_d14c_topsoil[i,ind] = isam.avg_wt_topsoil(soccm_test[ind].as_matrix()[i,], d14cm_test[ind].as_matrix()[i,])
        bulk_d14c_subsoil[i,ind] = isam.avg_wt_subsoil(soccm_test[ind].as_matrix()[i,], d14cm_test[ind].as_matrix()[i,])

    # Calculate the turnover time
    tau, cost = C14utils.cal_tau(bulk_d14c_topsoil[:,ind], sampleyr, 1, 0)
    tau_topsoil[:, ind] = tau[:,0]
    tau, cost = C14utils.cal_tau(bulk_d14c_subsoil[:,ind], sampleyr, 1, 0)
    tau_subsoil[:, ind] = tau[:,0]

    #tau[tau==2.00000000e+03] = np.float("nan")
    #data['tau'] = pd.Series(tau[:,0], index=data.index)
    #data.nodedepth = z*100
    #ttt=tau.reshape(255)

   
# Initialize record variables
con_topsoil = np.zeros((3, 5))
con_subsoil = np.zeros((3, 5))
sen_topsoil = np.zeros((24, 5))
sen_subsoil = np.zeros((24, 5))
tmp_topsoil = []
tmp_subsoil = []

# Calculate the sensitivity range of SOC, D14C and tau
for siteid, ind in site.items():   
    # top soil is the first 5 layer
    # subsoil is the 6th, 7th and part of 8th soil layer (30% depth)
    
    ## Case: aa
    # Get idxs
    idx_u = sen_exp.get("aa_u")
    idx_l = sen_exp.get("aa_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    ## Case: bb
    idx_u = sen_exp.get("bb_u")
    idx_l = sen_exp.get("bb_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())    
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    ## Case: bd
    idx_u = sen_exp.get("bd_u")
    idx_l = sen_exp.get("bd_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    # control
    idx = sen_exp.get("control")
    con_topsoil[0,ind] = isam.agg_topsoil(soccm_test[ind].as_matrix()[idx,:])
    con_subsoil[0,ind] = isam.agg_subsoil(soccm_test[ind].as_matrix()[idx,:])
    con_topsoil[1,ind] = isam.avg_wt_topsoil(soccm_test[ind].as_matrix()[idx,:], d14cm_test[ind].as_matrix()[idx,:])
    con_subsoil[1,ind] = isam.avg_wt_subsoil(soccm_test[ind].as_matrix()[idx,:], d14cm_test[ind].as_matrix()[idx,:])
    con_topsoil[2,ind] = tau_topsoil[idx, ind]
    con_subsoil[2,ind] = tau_subsoil[idx, ind]
    
    ## Case: d
    idx_u = sen_exp.get("d_u")
    idx_l = sen_exp.get("d_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    ## Case: dep
    idx_u = sen_exp.get("dep_u")
    idx_l = sen_exp.get("dep_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    ## Case: q 
    idx_u = sen_exp.get("q_u")
    idx_l = sen_exp.get("q_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    ## Case: s
    idx_u = sen_exp.get("s_u")
    idx_l = sen_exp.get("s_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    ## Case: gamma
    idx_u = sen_exp.get("gamma_u")
    idx_l = sen_exp.get("gamma_l")
    # SOC
    rgtop, rgsub = uq.get_sensitivity_range_SOC(idx_u, idx_l, soccm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # D14C
    rgtop, rgsub = uq.get_sensitivity_range_D14C(idx_u, idx_l, soccm_test[ind].as_matrix(), d14cm_test[ind].as_matrix())
    tmp_topsoil.append(rgtop)
    tmp_subsoil.append(rgsub)
    # tau
    tmp_topsoil.append((tau_topsoil[idx_u, ind] - tau_topsoil[idx_l, ind])/2.)
    tmp_subsoil.append((tau_subsoil[idx_u, ind] - tau_subsoil[idx_l, ind])/2.)
    
    sen_topsoil[:,ind] = tmp_topsoil
    sen_subsoil[:,ind] = tmp_subsoil
    
    tmp_topsoil=[]
    tmp_subsoil=[]
       
# Record the results into pandas dataframe and csv file
colname=['SOC', 'D14C', 'Turnover', 'aa_SOC', 'aa_D14C', 'aa_Turnover', 'bb_SOC', 'bb_D14C', 'bb_Turnover', 
         'bd_SOC', 'bd_D14C', 'bd_Turnover', 'd_SOC', 'd_D14C', 'd_Turnover', 'dep_SOC', 'dep_D14C', 'dep_Turnover',
        'q_SOC', 'q_D14C', 'q_Turnover', 's_SOC', 's_D14C', 's_Turnover', 'gamma_SOC', 'gamma_D14C', 'gamma_Turnover']
idx = ['43 topsoil', '110 topsoil', '143 topsoil', '146 topsoil', '197 topsoil', 
       '43 subsoil', '110 subsoil', '143 subsoil', '146 subsoil', '197 subsoil']
tb_topsoil = np.concatenate((con_topsoil, sen_topsoil),0)
tb_subsoil = np.concatenate((con_subsoil, sen_subsoil),0)
tb = np.concatenate((tb_topsoil, tb_subsoil),1)
outdf = pd.DataFrame(data=np.transpose(tb), index=idx, columns=colname)
outdf.to_csv('out/Table_sensitivity_results.csv')



 -----  obs: 0  ------ 



 -----  obs: 1  ------ 



 -----  obs: 2  ------ 



 -----  obs: 3  ------ 



 -----  obs: 4  ------ 



 -----  obs: 5  ------ 



 -----  obs: 6  ------ 



 -----  obs: 7  ------ 



 -----  obs: 8  ------ 



 -----  obs: 9  ------ 



 -----  obs: 10  ------ 



 -----  obs: 11  ------ 



 -----  obs: 12  ------ 



 -----  obs: 13  ------ 



 -----  obs: 14  ------ 



 -----  obs: 15  ------ 



 -----  obs: 16  ------ 



 -----  obs: 17  ------ 



 -----  obs: 0  ------ 



 -----  obs: 1  ------ 



 -----  obs: 2  ------ 



 -----  obs: 3  ------ 



 -----  obs: 4  ------ 



 -----  obs: 5  ------ 



 -----  obs: 6  ------ 



 -----  obs: 7  ------ 



 -----  obs: 8  ------ 



 -----  obs: 9  ------ 



 -----  obs: 10  ------ 



 -----  obs: 11  ------ 



 -----  obs: 12  ------ 



 -----  obs: 13  ------ 



 -----  obs: 14  ------ 



 -----  obs: 15  ------ 



 -----  obs: 16  ------ 



 -----  obs: 17  ------ 



 -----  obs

isamcalc_lib.py:204: RuntimeWarning: invalid value encountered in double_scalars
  wt[i] =  weight[(i+5)] / weight_tot
isamcalc_lib.py:206: RuntimeWarning: invalid value encountered in double_scalars
  wt[i] = 0.3*weight[(i+5)] / weight_tot
/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)





 -----  obs: 1  ------ 



 -----  obs: 2  ------ 



 -----  obs: 3  ------ 



 -----  obs: 4  ------ 



 -----  obs: 5  ------ 



 -----  obs: 6  ------ 



 -----  obs: 7  ------ 



 -----  obs: 8  ------ 



 -----  obs: 9  ------ 



 -----  obs: 10  ------ 



 -----  obs: 11  ------ 



 -----  obs: 12  ------ 



 -----  obs: 13  ------ 



 -----  obs: 14  ------ 



 -----  obs: 15  ------ 



 -----  obs: 16  ------ 



 -----  obs: 17  ------ 



 -----  obs: 0  ------ 



 -----  obs: 1  ------ 



 -----  obs: 2  ------ 



 -----  obs: 3  ------ 



 -----  obs: 4  ------ 



 -----  obs: 5  ------ 



 -----  obs: 6  ------ 



 -----  obs: 7  ------ 



 -----  obs: 8  ------ 



 -----  obs: 9  ------ 



 -----  obs: 10  ------ 



 -----  obs: 11  ------ 



 -----  obs: 12  ------ 



 -----  obs: 13  ------ 



 -----  obs: 14  ------ 



 -----  obs: 15  ------ 



 -----  obs: 16  ------ 



 -----  obs: 17  ------ 



 -----  obs: 0  ------ 



 -----  o